In [6]:
from gekko import GEKKO
import time
import numpy as np

#### Testing Gekko optimisation with arbitray class method 'prob', arbitray additional constraint function, and other settings from here: https://gekko.readthedocs.io/en/latest/quick_start.html#objectives

In [23]:
class prob:
    
    def __init__(self, x, y):
        
        self.x = x
        self.y = y
        
    def get_p(self):
        return sum([self.x,self.x,self.x]) * self.y**2

In [31]:
def objective(x1, x2, x3, x4):
    
    p = prob(x1, x2)
    #return np.sin(x1 + x2) + (x3 / x4)
    return p.get_p() + (x3 / x4)

def c_1(a, b):
    return a - b

In [32]:
#Initialize Model
m = GEKKO()

#define parameter
eq = m.Param(value=40)

#initialize variables
x1,x2,x3,x4 = [m.Var(lb=1, ub=5) for i in range(4)]

#initial values
x1.value = 1
x2.value = 5
x3.value = 5
x4.value = 1

#Equations
m.Equation(x1*x2*x3*x4>=25)
m.Equation(x1**2+x2**2+x3**2+x4**2==eq)
m.Equation(c_1(x2, x4)>=0 )

#Objective
#m.Obj(x1*x4*(x1+x2+x3)+x3)
m.Obj(objective(x1,x2,x3,x4))

#Set global options
m.options.IMODE = 3 #steady state optimization

#Solve simulation
m.solve()

#Results
print('')
print('Results')
print('x1: ' + str(x1.value))
print('x2: ' + str(x2.value))
print('x3: ' + str(x3.value))
print('x4: ' + str(x4.value))

apm 94.173.196.142_gk_model8 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            7
   Intermediates:            0
   Connections  :            0
   Equations    :            4
   Residuals    :            4
 
 Number of state variables:              6
 Number of total equations: -            3
 Number of slack variables: -            2
 ---------------------------------------
 Degrees of freedom       :              1
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

#### This seems to work. 

Our additional requirements will be:
- optimise expected_value (prob * risk)
- vector/list of parameters (5 skill entries for each worker -> length = 5*N)
- bound contraints [0,1] for each element
- sum of worker contributions (5 elements) <= how many units they have free to contribute 
- departmental workload is met (This is the hard one because it is dynamic: the constraint function will have to be re-evaluated with each assignment tested by the otpimisation routin. *Can Gekko handle this?* - it appears so from the above, but if not we may need to approximate the workload constraint.)

In [33]:
import sys, os
MODEL_DIR = os.path.realpath(os.path.dirname('..\superscript_model'))
sys.path.append(os.path.normpath(MODEL_DIR))

In [62]:
from superscript_model import model
from superscript_model.utilities import Random
from superscript_model.project import Project

In [59]:
STEPS = 15

abm = model.SuperScriptModel(worker_count=60, 
                             new_projects_per_timestep=2, 
                             worker_strategy = "Stake",
                             organisation_strategy = 'Basic')

In [60]:
abm.run_model(STEPS)

In [61]:
tracked = abm.datacollector.get_model_vars_dataframe()
tracked.head(STEPS)

,ActiveProjects,RecentSuccessRate,SuccessfulProjects,FailedProjects,NullProjects,WorkersOnProjects,WorkersWithoutProjects,WorkersOnTraining,AverageTeamSize,AverageSuccessProbability,AverageWorkerOvr,AverageTeamOvr,WorkerTurnover,ProjectLoad,TrainingLoad,DeptLoad,Slack,ProjectsPerWorker
0,2,0.000000,0.0,0.0,0,6,54,0,5.000000,0.000000,48.951417,20.928318,0.0,0.025000,0.000000,0.1,0.875000,0.116667
1,2,0.000000,0.0,0.0,1,6,54,0,5.000000,0.000000,48.497455,20.928318,0.0,0.025000,0.000000,0.1,0.875000,0.116667
2,2,0.000000,0.0,0.0,1,6,54,0,5.000000,0.000000,48.048033,20.928318,0.0,0.025000,0.000000,0.1,0.875000,0.116667
3,6,0.000000,0.0,0.0,0,22,38,0,6.333333,0.158725,47.713210,45.052762,0.0,0.170000,0.000000,0.1,0.730000,0.583333
4,6,0.000000,0.0,2.0,0,24,36,0,6.500000,0.175725,47.430664,50.675776,0.0,0.201667,0.000000,0.1,0.698333,0.650000
5,7,0.000000,0.0,0.0,0,26,34,0,6.571429,0.176007,47.198223,53.464343,0.0,0.226667,0.000000,0.1,0.673333,0.766667
6,8,0.000000,0.0,0.0,1,26,34,0,6.625000,0.189918,46.968106,52.884499,0.0,0.270000,0.000000,0.1,0.630000,0.883333
7,9,0.000000,0.0,2.0,0,30,30,0,6.666667,0.204353,46.555145,52.418440,0.0,0.290000,0.000000,0.1,0.610000,0.966667
8,7,0.088889,1.0,2.0,1,28,32,0,6.571429,0.150644,46.486724,48.584899,0.0,0.183333,0.000000,0.1,0.716667,0.733333
9,6,0.061111,0.0,2.0,1,29,29,2,7.000000,0.146135,51.977561,52.315154,21.0,0.188333,0.033333,0.1,0.678333,0.666667


#### Now, with the current state of the system, we will try creating a new project and finding its optimal team allocation:

In [64]:
project = Project(abm.inventory, 
                  abm.inventory.index_total + 1,
                  project_length=5,
                  start_time_offset=abm.inventory.get_start_time_offset(),
                  start_time=abm.schedule.steps)

In [68]:
print(project.requirements.to_string())

{
    "risk": 25,
    "creativity": 1,
    "flexible_budget": true,
    "budget": 95.0,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 21,
    "hard_skills": {
        "A": {
            "level": 4,
            "units": 6
        },
        "B": {
            "level": 3,
            "units": 6
        },
        "C": {
            "level": 4,
            "units": 6
        },
        "D": {
            "level": 3,
            "units": 2
        },
        "E": {
            "level": 4,
            "units": 1
        }
    }
}


In [72]:
bid_pool = abm.inventory.team_allocator.strategy.invite_bids(project)
print(len(bid_pool))

35


In [79]:
m = GEKKO()
x = m.Array(m.Var,(len(bid_pool) * 5))

initial_guess = [0 for i in range(len(x))]
i = 0
for i, xi in enumerate(x):
    xi.value = initial_guess[i]
    xi.lower = 0
    xi.upper = 1